# Session 2: Variational Quantum Optimization

In the last session we saw how to take an optimization problem written in DOcplex, map it to Qiskit's `QuadraticProgram` object, re-write the quadratic program as a QUBO, and map the QUBO on to an Ising Hamiltonian.

In this session, we'll discuss variational algorithms for estimating the ground state energy (and ground state) of the corresponding Ising Hamiltonian, which in turn gives us an estimate of the optimal value of the objective function, and the bitstring(s) which provide an optimal solution.

We'll start by discussing the Variational Quantum Eigensolver, then Trotterized annealing, and the Quantum Approximate Optimization Algorithm.

## From last time: MaxCut on a graph

In [ ]:
# Some standard code imports
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np

# For drawing graphs
import networkx as nx

# Qiskit imports
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization import QuadraticProgram

# auxilliary function to plot graphs
def plot_result(G, x):
    colors = ['r' if x[i] == 0 else 'b' for i in range(n)]
    pos, default_axes = nx.spring_layout(G), plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, pos=pos)

In [ ]:
#-------------------------MAKING THE GRAPH---------------------------#
# Create graph
G = nx.Graph()

# Add 5 nodes
n = 5
G.add_nodes_from(range(n))

# Add edges: tuple is (i,j,weight) where (i,j) is the edge
edges = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0), (2, 4, 1.0), (3, 4, 1.0)]
G.add_weighted_edges_from(edges)

#-----------------------WRITING A DOCPLEX MODEL--------------------#
# Import a model from DOcplex
from docplex.mp.model import Model

# Name the model
mdl = Model('MaxCut')

# Add a binary variable to the model for each node in the graph
x = mdl.binary_var_list('x{}'.format(i) for i in range(n))

# Define the objective function
objective = mdl.sum([ w * (x[i] + x[j] - 2*x[i]*x[j]) for (i, j, w) in edges])

# And let's maximize it!
mdl.maximize(objective)

# Add an equality constraint
b = 2
mdl.add_constraint(mdl.sum(x) == b)

#--------------------CONVERSION TO ISING, VIA QUADRATICPROGRAM-----#
# Set up the quadratic program
qp = QuadraticProgram()

# Put the model inside it
qp.from_docplex(mdl)

# Convert the program to a QUBO
qp_eq = QuadraticProgramToQubo(penalty=10).convert(qp)

H, offset = qp_eq.to_ising()

In [ ]:
H_matrix = np.real(H.to_matrix())

#Get the set of basis states which have the lowest energy
opt_indices = list(np.where(H_matrix.diagonal() == min(H_matrix.diagonal())))[0]
plt.figure(figsize=(12, 5))

print('Minimum energy for Hamiltonian: {0}'.format(min(H_matrix.diagonal())))
# Plot the expectation value of the energy of different basis states,
# and color those basis states which would have the lowest energy
plt.bar(range(2**n), H_matrix.diagonal())
plt.bar(opt_indices, H_matrix.diagonal()[opt_indices], color='g')
plt.xticks(range(2**n), ['('+str(i)+') {0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

## The Variational Quantum Eigensolver (VQE)


For VQE, the minimization over all $|\psi\rangle$ is replaced by a minimization over a parametrized subset $|\psi(\theta)\rangle$:
<br>

$$
\min_{\theta} \langle \psi(\theta) |H| \psi(\theta) \rangle
$$

Let's use [Qiskit's Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) to set up a VQE ansatz.

We'll use the [`RealAmplitudes` ansatz](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RealAmplitudes.html#qiskit.circuit.library.RealAmplitudes).

In [ ]:
from qiskit.circuit.library import RealAmplitudes

# Set up an ansatz with 5 qubits, and 1 repretition of the pattern.
qc = RealAmplitudes(5, reps=1)
qc.draw(output='mpl')

We can see the pattern of $RY$-gates, then all-to-all entangling gates, followed by another layer of $RY$-gates.

This ansatz has 10 parameters (5 qubits * 2 paramerized $RY$-gates per qubit).

Given this ansatz, let's do VQE with it.

In [ ]:
# Run VQE
from qiskit.aqua.algorithms import VQE

# We put in the Hamiltonian encoding our optimization problem,
# plus the quantum circuit for the ansatz,
# plus a "quantum_instance" that tells Qiskit what backend to use to run VQE
vqe = VQE(H, qc, quantum_instance=Aer.get_backend('statevector_simulator'))
result = vqe.run()
print('Estimated optimal value:', np.round(result.eigenvalue, decimals=4))

Let's take a look at which bitstring(s) we're most likely to get if we were to measure the optimized ansatz.

In [ ]:
# Compute probabilities
probabilities = np.abs(result.eigenstate)**2

opt_probs = probabilities[opt_indices]
print('Probability of observing an optimal bitstring: {0}'.format(np.sum(opt_probs)))
# Plot probabilities
plt.figure(figsize=(12, 5))
plt.bar(range(2**n), probabilities)
plt.bar(opt_indices, opt_probs , color='g')
plt.xticks(range(2**n), ['('+str(i)+') {0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()

## Towards QAOA: Trotterized annealing on 1 qubit

Suppose

$$
H_C = \sigma_Z = 
\left(\begin{array}{cc}
1 & 0 \\ 0 & -1
\end{array}\right)
$$

with <font color="blue">groundstate $|1\rangle$ and optimal value -1</font>.
Then,

$$
H_t = \frac{t}{T} \sigma_Z - (1 - \frac{t}{T}) \sigma_X.
$$

The annealing process can then be approximated by setting

$$ 
|\psi_{t+1}\rangle = e^{-iH_t \Delta t}|\psi_{t}\rangle
$$

for a small time step $\Delta t$. The matrices $\sigma_Z$ and $\sigma_X$ do not commute, however, we can trotterize, i.e., we first apply $e^{-i \frac{t}{T} H_C \Delta t}$ and then $e^{-i (1 - \frac{t}{T}) H_X \Delta t}$, and for sufficiently small $\Delta t$ only introduce a small error.

Thus, we have

$$
e^{-i \frac{t}{T} H_C \Delta t} = e^{-i \sigma_Z \frac{\gamma_t}{2}} = R_Z(\gamma_t),
$$

and

$$
e^{-i (1 - \frac{t}{T}) H_X \Delta t} = e^{-i \sigma_X \frac{\beta_t}{2}} = R_X(\beta_t)
$$

for rotation angles $\gamma_t = 2 t/T \Delta t$ and $\beta_t = -2(1 - t/T) \Delta t$.

Let's set up a parameterized quantum circuit for this. We'll then construct an annealing schedule, and see what happens.

In [ ]:
from qiskit.circuit import Parameter
gamma, beta = Parameter('gamma'), Parameter('beta')

# This circuit would be 1 time step
qc = QuantumCircuit(1)
qc.h(0)
qc.barrier()
qc.rz(gamma, 0)
qc.rx(beta, 0)
qc.barrier()
qc.draw(output='mpl')

In [ ]:
def construct_schedule(T, N):
    delta_t = T/N
    gammas, betas = [], []  # H_C, H_X parameters
    for i in range(N+1):
        t = i * delta_t
        gammas += [ 2 * delta_t * t/T ]  # H_C
        betas += [ -2 * delta_t * (1 - t/T) ]  # H_X
    return gammas, betas

In [ ]:
T = 5
N = 10
gammas, betas = construct_schedule(T, N)

For each time step (i.e., $(\gamma, \beta)$ pair), we need to add a layer to the quantum circuit.

We'll build up the circuit timestep by timestep, and also simulate it using the fact Qiskit's `Statevector` object can compute the statevector given the instructions in the circuit.

In [ ]:
# Track probabilities during trotterized annealing
probabilities = np.zeros((2, N+1))

# Set up the circuit
qc = QuantumCircuit(1)
qc.h(0)
qc.barrier()
# Do the evolution
for i, (gamma, beta) in enumerate(zip(gammas, betas)):
    qc.rz(gamma, 0)
    qc.rx(beta, 0)
    qc.barrier()
    #Simulate the circuit, and store the probability of |0> and |1> at each timestep
    probabilities[:, i] = Statevector.from_instruction(qc).probabilities()

In [ ]:
# This is the full circuit.
qc.draw(output='mpl')

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(np.linspace(0, T, N+1), probabilities[1, :], 'gd-', label=r'$|1\rangle$')
plt.plot(np.linspace(0, T, N+1), probabilities[0, :], 'bo-', label=r'$|0\rangle$')
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel('time $t$', fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('probabilities', fontsize=14);

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(np.linspace(0, T, N+1), probabilities[0, :] - probabilities[1, :], 'gd-')
plt.xticks(fontsize=14)
plt.xlabel('time $t$', fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('objective value', fontsize=14);

During the evolution of the state, the probability the state is in $|1\rangle$ increases.

## Towards QAOA: Trotterized Annealing for our MaxCut problem

Let's see how we can use Trotterized annealing to tackle our MaxCut problem.

To do this, we'll use Qiskit Aqua's `QAOAVarForm` as a helper function to set up the circuit for Trotterized evolution.

In [ ]:
from qiskit.aqua.algorithms.minimum_eigen_solvers.qaoa.var_form import QAOAVarForm

In [ ]:
# Construct parameters from annealing schedule
T = 10
N = 20
gammas, betas = construct_schedule(T, N)

# Construct variational form
# Note: Here, the number of layers, p, is equal to the number of timesteps
var_form = QAOAVarForm(H, N+1)

# Create the quantum circuit
qc = var_form.construct_circuit(gammas + betas)

In [ ]:
# Let's draw this thing!
qc.draw(output='mpl')

In [ ]:
# Compute the statevector
sv = Statevector.from_instruction(qc)

# Get the associated probabilities of the individual bitstrings
probabilities = sv.probabilities()

In [ ]:
opt_probs = probabilities[opt_indices]

print('Probability of observing an optimal bitstring: {0}'.format(np.round(np.sum(opt_probs), 4)))

# Plot probabilities
plt.figure(figsize=(12, 5))
plt.bar(range(2**n), probabilities)
plt.bar(opt_indices, opt_probs, color='g')
plt.xticks(range(2**n), ['('+str(i)+') {0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)

Bitstrings which correspond to optimal solutions do not appear very often...

## QAOA for our MaxCut problem

In the previous example, we manually set an annealing schedule. Here, we're going to use numerical optimization of the $\gamma, \beta$ parameters to determine which angles are the best. Here, "best" is going to mean "maximize our objective function".

We'll use a $p=1$ ansatz for simplicty, which requires 2 parameters.

In [ ]:
# Two parameters for the ansatz
gamma, beta = Parameter('gamma'), Parameter('beta')

In [ ]:
# Let's look at what the ansatz looks like.
p = 1
var_form = QAOAVarForm(H, p)
qc = var_form.construct_circuit([gamma, beta])

qc.draw(output='mpl')

In order to tune the parameters, we need an optimizer. Here, we'll use the COBYLA optimizer, which is exposed via a [wrapped function call in Qiskit](https://qiskit.org/documentation/stubs/qiskit.aqua.components.optimizers.COBYLA.html).

In [ ]:
# Import the COBYLA optimizer
from qiskit.aqua.components.optimizers import COBYLA

# Import the QAOA algorithm
from qiskit.aqua.algorithms import QAOA

In [ ]:
# Set the optimizer
optimizer = COBYLA()

# Set up QAOA
qaoa_mes = QAOA(H, p=1, optimizer=optimizer, quantum_instance=Aer.get_backend('statevector_simulator'))

In [ ]:
# Run QAOA
result = qaoa_mes.run()

Let's take a look at the results.

In [ ]:
print('optimal params:      ', result.optimal_parameters)
print('optimal value:       ', result.optimal_value)

In [ ]:
# Compute the probabilities given the result
probabilities = np.abs(result['eigenstate'])**2

print('Probability of observing an optimal bitstring: {0}'.format(np.round(np.sum(probabilities[opt_indices]), 4)))

# Plot probabilities
plt.figure(figsize=(12, 5))
plt.bar(range(2**n), probabilities)
plt.bar(opt_indices, probabilities[opt_indices], color='g')
plt.xticks(range(2**n), ['('+str(i)+') {0:05b}'.format(i) for i in range(2**n)], rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.show()


## Wrapping QAOA

In the above example, we used QAOA to solve our MaxCut problem. But the `result` object we got back talks about QAOA parameters, an eigenstate, etc. -- _not_ objective  functions of partitions of our graph!

To rectify this, we can _wrap_ QAOA using Qiskit Optimization's `MinimumEigenOptimizer` object. This will provide us with a simple interface for solving our MaxCut problem.

In [ ]:
from qiskit.optimization.algorithms import MinimumEigenOptimizer

In [ ]:
# construct QAOA as Minimum Eigensolver
qaoa_mes = QAOA(p=1, optimizer=optimizer, quantum_instance=Aer.get_backend('statevector_simulator'))

# construct Minimum Eigen Optimizer based on QAOA
qaoa = MinimumEigenOptimizer(qaoa_mes)

# solve Quadratic Program
# Notice we can give QAOA the original quadratic program,
# **not** the one we re-wrote by changing the equality constraint
# into a penalty!
result = qaoa.solve(qp)

In [ ]:
print(result)
plot_result(G, result.x)